In [ ]:
import carla
import numpy as np
import cv2


In [ ]:
import matplotlib.pyplot as plt
import random
import time
from PIL import Image
import pandas as pd


In [ ]:
client=carla.Client("localhost",2000)
world=client.get_world()


In [ ]:
steer_angle_info2=[]
throttle_in_info2=[]
brake_in_info2=[]
save_img_info2=[]

In [ ]:
steer_angle_info3=[]
throttle_in_info3=[]
brake_in_info3=[]
save_img_info3=[]

In [ ]:
allspawn=[]

In [ ]:
blueprint_library = world.get_blueprint_library()
spawnpoints=world.get_map().get_spawn_points()

In [ ]:
ar=blueprint_library.find("vehicle.mercedes.coupe_2020")
while True:
    transforms2=random.choice(spawnpoints)
    if transforms2 not in allspawn:
        allspawn.append(transforms2)
        break
car2=world.try_spawn_actor(ar,allspawn[0])

In [ ]:
ar=blueprint_library.find("vehicle.mercedes.coupe_2020")
while True:
    transforms2=random.choice(spawnpoints)
    if transforms2 not in allspawn:
        allspawn.append(transforms2)
        break
car3=world.try_spawn_actor(ar,allspawn[1])

In [ ]:
spectator=world.get_spectator()
x=carla.Transform(car2.get_transform().transform(carla.Location(x=-0.4,z=1.8)),car2.get_transform().rotation)
spectator.set_transform(x)

In [ ]:
cam2=blueprint_library.find("sensor.camera.rgb")
cam2.set_attribute("image_size_x","1920")
cam2.set_attribute("image_size_y","1080")
cam2.set_attribute("fov","110")
cam2.set_attribute("sensor_tick",".5")
transforms=carla.Transform(carla.Location(z=1.8,x=-.2))
camera2=world.spawn_actor(cam2,transforms,attach_to=car2)
time.sleep(.2)



In [ ]:
cam3=blueprint_library.find("sensor.camera.rgb")
cam3.set_attribute("image_size_x","1920")
cam3.set_attribute("image_size_y","1080")
cam3.set_attribute("fov","110")
cam3.set_attribute("sensor_tick",".5")
transforms=carla.Transform(carla.Location(z=1.8,x=-.2))
camera3=world.spawn_actor(cam3,transforms,attach_to=car3)
time.sleep(.2)



In [ ]:
car2.set_autopilot()
time.sleep(.2)

In [ ]:
car3.set_autopilot()
time.sleep(.2)

In [ ]:
def show_me(image,camera_data):
    r=np.array(image.raw_data)
    r1=r.reshape((1080,1920,4))
    r2=r1[:,:,:3]
    im=cv2.cvtColor(r2,cv2.COLOR_BGR2RGB)
    im=Image.fromarray(im)
    A=image.frame
    im.save('pokemon3/%06d.png'%A)
    save_img_info3.append('pokemon3/%06d.png'%A)
    steer_angle_info3.append(car3.get_control().steer)
    throttle_in_info3.append(car3.get_control().throttle)
    brake_in_info3.append(car3.get_control().brake)
    camera_data["zs"]=r2
    

In [ ]:
def show_me2(image):
    r=np.array(image.raw_data)
    r1=r.reshape((1080,1920,4))
    r2=r1[:,:,:3]
    im=cv2.cvtColor(r2,cv2.COLOR_BGR2RGB)
    im=Image.fromarray(im)
    A=image.frame
    im.save('pokemon3/%06d.png'%A)
    save_img_info2.append('pokemon3/%06d.png'%A)
    steer_angle_info2.append(car2.get_control().steer)
    throttle_in_info2.append(car2.get_control().throttle)
    brake_in_info2.append(car2.get_control().brake)

    

In [ ]:
camera2.listen(lambda image:show_me2(image))
time.sleep(.1)


In [ ]:
image_w=cam3.get_attribute("image_size_x").as_int()
image_h=cam3.get_attribute("image_size_y").as_int()
camera_data={"zs":np.zeros((image_h,image_w,3))}
camera3.listen(lambda image:show_me(image,camera_data))
time.sleep(.1)



In [ ]:
def updater(image,lis1,lis2,lis3):
    if len(image)>len(lis1):
        image.pop()
    elif len(lis1)>len(lis2):
        image.pop()
        lis1.pop()
    elif len(lis2)>len(lis3):
        image.pop()
        lis1.pop()
        lis2.pop()                    

In [ ]:
cv2.namedWindow("RGB CAMERA",cv2.WINDOW_AUTOSIZE)
cv2.waitKey(1)
while True:
    cv2.imshow("RGB CAMERA",camera_data["zs"])
    if cv2.waitKey(1) & 0xFF ==ord("Q"):
        camera2.destroy()
        camera3.destroy()
        car2.destroy()
        car3.destroy()
        updater(save_img_info2,throttle_in_info2,steer_angle_info2,brake_in_info2)
        df2=pd.DataFrame({"cam_path":save_img_info2,"Throttle":throttle_in_info2,"Steering":steer_angle_info2,"Braking":brake_in_info2})
        updater(save_img_info3,throttle_in_info3,steer_angle_info3,brake_in_info3)
        df3=pd.DataFrame({"cam_path":save_img_info3,"Throttle":throttle_in_info3,"Steering":steer_angle_info3,"Braking":brake_in_info3})
        df2.to_csv(r"D:\\Carla Dataset formation\\data5.csv")
        df3.to_csv(r"D:\\Carla Dataset formation\\data6.csv")
        
        break
cv2.destroyAllWindows()